#### Лабораторная работа № 1. Разработка нейросетевых функций. Операция Convolution 2D
#### БВТ2003 Никитин Степан Романович

In [39]:
import torch
import numpy as np
from typing import Union

In [40]:
class Conv2D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        """
        Конструктор класса Conv2D
        """
        self.in_channels = in_channels     # количество входных каналов (каналов входного тензора)
        self.out_channels = out_channels   # количество выходных каналов (каналов результата свертки)
        self.kernel_size = kernel_size     # размер ядра свертки (предполагается квадратное ядро)
        self.stride = stride               # шаг свертки

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        # инициализация весов (ядер свертки)
        self.weights = np.random.randn(out_channels, in_channels, *kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        """
        Метод для выполнения операции свертки
        """
        batch_size, in_channels, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        # проверка на корректные размеры входного тензора
        if input_height < kernel_size[0] or input_width < kernel_size[1]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # вычисление размеров результата свертки
        if isinstance(self.stride, int):
            stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 2:
            stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из двух целых чисел")

        output_height = (input_height - kernel_size[0]) // stride_h + 1
        output_width = (input_width - kernel_size[1]) // stride_w + 1

        # инициализация результата свертки
        result = np.zeros((batch_size, self.out_channels, output_height, output_width))

        # выполнение свертки
        for i in range(0, input_height - kernel_size[0] + 1, stride_h):
            for j in range(0, input_width - kernel_size[1] + 1, stride_w):
                # окно свертки
                window = input_tensor[:, :, i:i + kernel_size[0], j:j + kernel_size[1]]

                # вычисление свертки и добавление bias
                result[:, :, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3)) + self.bias

        return result

In [41]:
def run_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple], 
             padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    """
    Функция для выполнения теста операции свертки
    """
    # создание экземпляра Conv2D
    conv_layer = Conv2D(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    # создание тензора для теста
    input_tensor = np.random.randn(1, in_channels, 10, 10)

    # выполнение свертки
    result_conv2d = conv_layer.forward(input_tensor)

    # преобразование результатов в формат torch.Tensor для сравнения с PyTorch
    input_tensor_torch = torch.tensor(input_tensor)
    output_tensor_torch = torch.nn.functional.conv2d(input_tensor_torch, 
                                                      torch.tensor(conv_layer.weights), 
                                                      bias=torch.tensor(conv_layer.bias), 
                                                      stride=stride, 
                                                      padding=padding, 
                                                      dilation=dilation, 
                                                      groups=groups).numpy()

    # обрезание результат из Conv2D, чтобы сделать размеры одинаковыми
    min_height = min(result_conv2d.shape[2], output_tensor_torch.shape[2])
    min_width = min(result_conv2d.shape[3], output_tensor_torch.shape[3])
    result_conv2d = result_conv2d[:, :, :min_height, :min_width]
    output_tensor_torch = output_tensor_torch[:, :, :min_height, :min_width]

    # проверка совпадения результатов
    are_results_equal = np.allclose(result_conv2d, output_tensor_torch)
    
    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nРезультат свертки совпадает: {are_results_equal}")
    print(f"\n{'-'*75}\n")

In [42]:
def test_params() -> None:
    """
    Функция для тестирования различных параметров операции свертки
    """
    # параметры тестирования для первой группы
    run_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

    # параметры тестирования для второй группы
    run_test(in_channels=8, out_channels=4, kernel_size=2, stride=(2, 3), padding=0, dilation=1, groups=1, bias=True, padding_mode='replicate')

    # параметры тестирования для третьей группы
    run_test(in_channels=5, out_channels=32, kernel_size=(3, 2), stride=(4, 3), padding=1, dilation=2, groups=1, bias=True, padding_mode='zeros')


In [43]:
# параметры тестирования для дополнительной группы
input_tensor = np.random.randn(1, 3, 5, 5)
conv_layer = Conv2D(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2))
output_tensor = conv_layer.forward(input_tensor)

print(f"Результат свертки (Conv2D):\n {output_tensor}")

Результат свертки (Conv2D):
 [[[[-3.82491817  1.56569356]
   [ 5.58712545 -5.82472669]]

  [[-5.29676349  2.95685182]
   [-1.85297289  1.30892066]]]]


In [44]:
# преобразование результатов в формат torch.Tensor для сравнения с PyTorch
input_tensor_torch = torch.tensor(input_tensor)
output_tensor_torch = torch.nn.functional.conv2d(input_tensor_torch, 
                                                  torch.tensor(conv_layer.weights), 
                                                  bias=torch.tensor(conv_layer.bias), 
                                                  stride=(2, 2)).numpy()

print(f"Результат свертки (PyTorch):\n {output_tensor_torch}")

Результат свертки (PyTorch):
 [[[[-3.82491817  1.56569356]
   [ 5.58712545 -5.82472669]]

  [[-5.29676349  2.95685182]
   [-1.85297289  1.30892066]]]]


In [45]:
test_params()

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: zeros

Результат свертки совпадает: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 8
  - out_channels: 4
  - kernel_size: 2
  - stride: (2, 3)
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: replicate

Результат свертки совпадает: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 5
  - out_channels: 32
  - kernel_size: (3, 2)
  - stride: (4, 3)
  - padding: 1
  - dilation: 2
  - groups: 1
  - bias: True
  - padding_mode: zeros

Результат свертки совпадает: False

---------------------------------------------------------------------------

